In [1]:
!pip install transformers

     |████████████████████████████████| 890kB 12.3MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
     |████████████████████████████████| 3.0MB 47.1MB/s 
     |████████████████████████████████| 1.1MB 52.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3cf99c1244e903ceb48f333c7d809dde9011bcb151a0296cb6316981b99180d8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences

import torch
import numpy as np

In [3]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
output_dir = '/content/drive/My Drive/BERT Fine-Tuning(0917)/'

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

In [6]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [7]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # -----------------------수정할 부분 추가해줘야함!-----------------------

    # -----------------------------------------------------------------------
    
    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [8]:
sentiment_dic = {
    0 : 'anger',
    1 : 'fear',
    2 : 'joy',
    3 : 'love',
    4 : 'neutral',
    5 : 'sadness',
    6 : 'surprise',
}

# ['anger', 'fear', 'joy', 'love', 'neutral', 'sadness', 'surprise']


In [10]:
logits = test_sentences(['최태성의 한국사를 들어 시험에 합격해서 기쁘다'])

print(logits)
print(sentiment_dic[np.argmax(logits)])

# label_encoder.inverse_transform(sentiment)

[[ 0.8600274  -1.2648774   1.2894237  -1.1729147   0.9290735  -0.77376294
  -0.26073506]]
joy


In [11]:
# a = input()

a = "ㅋㅋㅋㅋㅋㅋㅋ"

# ㅋㅋㅋ가 여러개면 ㅋㅋ로
a = a.replace("[ㄱ-ㅊ]|[ㅌㅍ]|[ㅏ-ㅣ]", "")
a = a.replace("[ㅎ]", " ㅋ ")
a = a.replace("[ㅋ]{2,}", " ㅋㅋ ")
a = a.replace("[.]", "")
# a = a.replace("[_ㅜㅠㅡㅓㅏ]","")
a = a.replace("&lt;|&gt;", "")
a = a.replace("[^\w]|_", " ")
a = a.replace("[ ]{2,}", " ")
a = a.replace("[아]{2,}",  "아아 ")
a = a.replace("[으]{2,}", " 으으 ")
a = a.replace("[하]{2,}", " 하하 ")
a[:10]
# text

print(len(a))
print(a)

print(a[:512])

7
ㅋㅋㅋㅋㅋㅋㅋ
ㅋㅋㅋㅋㅋㅋㅋ


In [ ]:
sequence_dic = {
    
}

In [34]:
logits = test_sentences([a])

print(logits)

print(np.argsort(logits)[0][-1])

# print(sentiment_dic[np.argmax(logits)])

print("1순위 : " + str(sentiment_dic[np.argsort(logits)[0][-1]]))
print("2순위 : " + str(sentiment_dic[np.argsort(logits)[0][-2]]))
print("3순위 : " + str(sentiment_dic[np.argsort(logits)[0][-3]]))

# label_encoder.inverse_transform(sentiment)


[[ 0.18664359 -1.1942226   1.0339078  -0.49703676 -0.3282728   0.934556
  -0.23835406]]
2
1순위 : joy
2순위 : sadness
3순위 : anger
